In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-addon-mpf

*推定QPU使用量：Heron r2プロセッサで約4分（注意：これは推定値です。実際の実行時間は異なる場合があります。）*

## 背景

このチュートリアルでは、マルチプロダクト公式（MPF）を使用して、実際に実行する最も深いTrotter回路で発生するTrotterエラーよりも低いTrotterエラーをオブザーバブルに対して達成する方法を示します。
MPFは、複数の回路実行の重み付き組み合わせを通じて、ハミルトニアンダイナミクスのTrotterエラーを低減します。ハミルトニアン$H$を用いた量子状態
$\rho(t)=e^{-i H t} \rho(0) e^{i H t}$のオブザーバブル期待値を求めるタスクを考えます。プロダクト公式（PF）を使用して時間発展$e^{-i H t}$を近似するには、以下を行います：

- ハミルトニアン$H$を$H=\sum_{a=1}^d F_a$と書きます。ここで$F_a$はエルミート演算子であり、対応する各ユニタリーは量子デバイス上で効率的に実装できます。
- 互いに可換でない項$F_a$を近似します。

すると、1次PF（Lie-Trotter公式）は以下のようになります：

$$S_1(t):=\prod_{a=1}^d e^{-i F_a t},$$

これは2次のエラー項$S_1(t)=e^{-i H t}+\mathcal{O}\left(t^{2}\right)$を持ちます。より高次のPF（Lie-Trotter-Suzuki公式）を使用することもできます。これらはより速く収束し、再帰的に定義されます：

$$S_2(t):=\prod_{a=1}^d e^{-i F_a t/2}\prod_{a=1}^d e^{-i F_a t/2}$$

$$S_{2 \chi}(t):= S_{2 \chi -2}(s_{\chi}t)^2 S_{2 \chi -2}((1-4s_{\chi})t)S_{2 \chi -2}(s_{\chi}t)^2,$$

ここで$\chi$は対称PFの次数であり、$s_p = \left( 4 - 4^{1/(2p-1)} \right)^{-1}$です。長時間の時間発展では、時間区間$t$を$k$個の区間（Trotterステップと呼ばれる）に分割し、各区間の持続時間$t/k$における時間発展を$\chi$次のプロダクト公式$S_{\chi}$で近似できます。したがって、$k$個のTrotterステップに対する次数$\chi$のPFによる時間発展演算子は以下のようになります：

$$ S_{\chi}^{k}(t) = \left[ S_{\chi} \left( \frac{t}{k} \right)\right]^k = e^{-i H t}+O\left(t \left( \frac{t}{k} \right)^{\chi} \right)$$

ここでエラー項はTrotterステップ数$k$およびPFの次数$\chi$とともに減少します。

整数$k \geq 1$とプロダクト公式$S_{\chi}(t)$が与えられたとき、近似的な時間発展状態$\rho_k(t)$は、プロダクト公式$S_{\chi}\left(\frac{t}{k}\right)$の$k$回の反復を$\rho_0$に適用することで得られます。

$$
\rho_k(t)=S_{\chi}\left(\frac{t}{k}\right)^k \rho_0 S_{\chi}\left(\frac{t}{k}\right)^{-k}
$$

$\rho_k(t)$はTrotter近似エラー ||$\rho_k(t)-\rho(t) ||$を持つ$\rho(t)$の近似です。$\rho(t)$のTrotter近似の線形結合を考えると：

$$
\mu(t) = \sum_{j}^{l} x_j \rho^{k_j}_{j}\left(\frac{t}{k_j}\right) + \text{some remaining Trotter error} \, ,
$$

ここで$x_j$は重み係数、$\rho^{k_j}_j$はプロダクト公式$S^{k_j}_{\chi}$（$k_j$個のTrotterステップを含む）で初期状態を発展させて得られる純粋状態に対応する密度行列、$j \in {1, ..., l}$はMPFを構成するPFの数のインデックスです。$\mu(t)$のすべての項は、同じプロダクト公式$S_{\chi}(t)$を基礎として使用します。
目標は、$\mu(t)$を見つけることで、$\|\mu(t)-\rho(t)\|$がさらに小さくなるようにし、||$\rho_k(t)-\rho(t) \|$を改善することです。

* $\mu(t)$は物理的な状態である必要はありません。$x_i$が正である必要がないためです。ここでの目標は、オブザーバブルの期待値のエラーを最小化することであり、$\rho(t)$の物理的な代替を見つけることではありません。
* $k_j$は回路の深さとTrotter近似のレベルの両方を決定します。$k_j$の値が小さいほど回路が短くなり、回路エラーは少なくなりますが、所望の状態に対するTrotter近似の精度は低下します。

ここでの重要な点は、$\mu(t)$によるTrotterの残留エラーが、単に最大の$k_j$値を使用した場合に得られるTrotterエラーよりも小さいということです。

この有用性は2つの観点から見ることができます：

1. 実行可能なTrotterステップの固定予算に対して、合計でより小さなTrotterエラーの結果を得ることができます。
2. 実行するには大きすぎるTrotterステップのターゲット数が与えられた場合、MPFを使用して、同様のTrotterエラーをもたらす、より浅い深さの回路のコレクションを見つけることができます。

## 要件

このチュートリアルを開始する前に、以下がインストールされていることを確認してください：

* Qiskit SDK v1.0以降、[可視化](https://docs.quantum.ibm.com/api/qiskit/visualization)サポート付き
* Qiskit Runtime v0.22以降（`pip install qiskit-ibm-runtime`）
* MPF Qiskit アドオン（`pip install qiskit_addon_mpf`）
* Qiskit アドオンユーティリティ（`pip install qiskit_addon_utils`）
* Quimbライブラリ（`pip install quimb`）
* Qiskit Quimbライブラリ（`pip install qiskit-quimb`）
* パッケージ間の互換性のためのNumpy v0.21（`pip install numpy==0.21`）

## パートI. 小規模な例

### MPFの安定性の探索

MPF状態$\mu(t)$を構成するTrotterステップ数$k_j$の選択には明確な制約がありません。しかし、$\mu(t)$から計算される期待値の不安定性を避けるため、これらは慎重に選択する必要があります。一般的な良いルールは、最小のTrotterステップ$k_{\text{min}}$を$t/k_{\text{min}} \lt 1$となるように設定することです。これについてや他の$k_j$値の選び方についてさらに学びたい場合は、[MPFのTrotterステップの選び方](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html)ガイドを参照してください。

以下の例では、さまざまな時間発展状態を使用して、一連の時間に対する磁化の期待値を計算することにより、MPF解の安定性を探索します。具体的には、対応するTrotterステップで実装された各近似的時間発展と、さまざまなMPFモデル（静的および動的係数）から計算された期待値を、時間発展したオブザーバブルの厳密値と比較します。まず、Trotter公式と発展時間のパラメータを定義しましょう。

In [1]:
import numpy as np

mpf_trotter_steps = [1, 2, 4]
order = 2
symmetric = False

trotter_times = np.arange(0.5, 1.55, 0.1)
exact_evolution_times = np.arange(trotter_times[0], 1.55, 0.05)

この例では、初期状態としてネール状態$\vert \text{Neel} \rangle = \vert 0101...01 \rangle$を使用し、時間発展を支配するハミルトニアンとして10サイトの直線上のハイゼンベルグモデルを使用します。

$$
\hat{\mathcal{H}}_{Heis} = J \sum_{i=1}^{L-1} \left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ Z_i Z_{(i+1)} \right) \, ,
$$

ここで$J$は最近接エッジの結合強度です。

In [2]:
from qiskit.transpiler import CouplingMap
from rustworkx.visualization import graphviz_draw
from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian
import numpy as np


L = 10

# Generate some coupling map to use for this example
coupling_map = CouplingMap.from_line(L, bidirectional=False)
graphviz_draw(coupling_map.graph, method="circo")

# Get a qubit operator describing the Heisenberg field model
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(1.0, 1.0, 1.0),
    ext_magnetic_field=(0.0, 0.0, 0.0),
)


print(hamiltonian)

SparsePauliOp(['IIIIIIIXXI', 'IIIIIIIYYI', 'IIIIIIIZZI', 'IIIIIXXIII', 'IIIIIYYIII', 'IIIIIZZIII', 'IIIXXIIIII', 'IIIYYIIIII', 'IIIZZIIIII', 'IXXIIIIIII', 'IYYIIIIIII', 'IZZIIIIIII', 'IIIIIIIIXX', 'IIIIIIIIYY', 'IIIIIIIIZZ', 'IIIIIIXXII', 'IIIIIIYYII', 'IIIIIIZZII', 'IIIIXXIIII', 'IIIIYYIIII', 'IIIIZZIIII', 'IIXXIIIIII', 'IIYYIIIIII', 'IIZZIIIIII', 'XXIIIIIIII', 'YYIIIIIIII', 'ZZIIIIIIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


The observable that we will be measuring is magnetization on a pair of qubits in the middle of the chain.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_sparse_list(
    [("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L
)
print(observable)

SparsePauliOp(['IIIIZZIIII'],
              coeffs=[1.+0.j])


測定するオブザーバブルは、チェーンの中央にある量子ビットのペアの磁化です。

In [4]:
from qiskit.circuit.library import XXPlusYYGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.optimization.collect_and_collapse import (
    CollectAndCollapse,
    collect_using_filter_function,
    collapse_to_operation,
)
from functools import partial


def filter_function(node):
    return node.op.name in {"rxx", "ryy"}


collect_function = partial(
    collect_using_filter_function,
    filter_function=filter_function,
    split_blocks=True,
    min_block_size=1,
)


def collapse_to_xx_plus_yy(block):
    param = 0.0
    for node in block.data:
        param += node.operation.params[0]
    return XXPlusYYGate(param)


collapse_function = partial(
    collapse_to_operation,
    collapse_function=collapse_to_xx_plus_yy,
)

pm = PassManager()
pm.append(CollectAndCollapse(collect_function, collapse_function))

Then we create the circuits implementing the approximate Trotter time-evolutions.

In [5]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


# Initial Neel state preparation
initial_state_circ = QuantumCircuit(L)
initial_state_circ.x([i for i in range(L) if i % 2 != 0])


all_circs = []
for total_time in trotter_times:
    mpf_trotter_circs = [
        generate_time_evolution_circuit(
            hamiltonian,
            time=total_time,
            synthesis=SuzukiTrotter(reps=num_steps, order=order),
        )
        for num_steps in mpf_trotter_steps
    ]

    mpf_trotter_circs = pm.run(
        mpf_trotter_circs
    )  # Collect XX and YY into XX + YY

    mpf_circuits = [
        initial_state_circ.compose(circuit) for circuit in mpf_trotter_circs
    ]
    all_circs.append(mpf_circuits)

In [6]:
mpf_circuits[-1].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/92dc20a7-0.avif" alt="Output of the previous code cell" />

次に、近似的なTrotter時間発展を実装する回路を作成します。

In [24]:
from copy import deepcopy
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


aer_sim = AerSimulator()
estimator = Estimator(mode=aer_sim)

mpf_expvals_all_times, mpf_stds_all_times = [], []
for t, mpf_circuits in zip(trotter_times, all_circs):
    mpf_expvals = []
    circuits = [deepcopy(circuit) for circuit in mpf_circuits]
    pm_sim = generate_preset_pass_manager(
        backend=aer_sim, optimization_level=3
    )
    isa_circuits = pm_sim.run(circuits)
    result = estimator.run(
        [(circuit, observable) for circuit in isa_circuits], precision=0.005
    ).result()
    mpf_expvals = [res.data.evs for res in result]
    mpf_stds = [res.data.stds for res in result]
    mpf_expvals_all_times.append(mpf_expvals)
    mpf_stds_all_times.append(mpf_stds)

We also calculate the exact expectation values for comparison.

In [ ]:
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

exact_expvals = []
for t in exact_evolution_times:
    # Exact expectation values
    exp_H = expm(-1j * t * hamiltonian.to_matrix())
    initial_state = Statevector(initial_state_circ).data
    time_evolved_state = exp_H @ initial_state

    exact_obs = (
        time_evolved_state.conj()
        @ observable.to_matrix()
        @ time_evolved_state
    ).real
    exact_expvals.append(exact_obs)

![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/92dc20a7-0.avif)

次に、Trotter回路から時間発展した期待値を計算します。

In [9]:
from qiskit_addon_mpf.static import setup_static_lse

lse = setup_static_lse(mpf_trotter_steps, order=order, symmetric=symmetric)

比較のために厳密な期待値も計算します。

In [10]:
lse.A

array([[1.      , 1.      , 1.      ],
       [1.      , 0.25    , 0.0625  ],
       [1.      , 0.125   , 0.015625]])

#### 静的MPF係数
静的MPFとは、$x_j$の値が発展時間$t$に依存しないものです。次数$\chi = 1$のPFが$k_j$個のTrotterステップを持つ場合を考えると、これは以下のように書けます：

$$ S_1^{k_j}\left( \frac{t}{k_j} \right)=e^{-i H t}+ \sum_{n=1}^{\infty} A_n \frac{t^{n+1}}{k_j^n}  $$

ここで$A_n$はハミルトニアンの分解における$F_a$項の交換子に依存する行列です。重要なのは、$A_n$自体は時間およびTrotterステップ数$k_j$に依存しないということです。したがって、重み$x_j$の線形結合を慎重に選ぶことで、$\mu(t)$に寄与する低次のエラー項を打ち消すことが可能です。$\mu(t)$の式における最初の$l-1$項（これらはTrotterステップ数が少ないため最大の寄与をもたらします）のTrotterエラーを打ち消すには、係数$x_j$は以下の方程式を満たす必要があります：

$$ \sum_{j=1}^l x_j = 1 $$
$$ \sum_{j=1}^{l-1} \frac{x_j}{k_j^{n}} = 0 $$

ここで$n=0, ... l-2$です。最初の方程式は構成された状態$\mu(t)$にバイアスがないことを保証し、2番目の方程式はTrotterエラーの打ち消しを保証します。高次のPFの場合、2番目の方程式は$ \sum_{j=1}^{l-1} \frac{x_j}{k_j^{\eta}} = 0 $となります。ここで$\eta = \chi + 2n$（対称PFの場合）または$\eta = \chi + n$（それ以外の場合）であり、$n=0, ..., l-2$です。結果として得られるエラー（参考文献[\[1\]](#references)、[\[2\]](#references)）は以下のようになります：

$$ \epsilon = \mathcal{O} \left( \frac{t^{l+1}}{k_1^l} \right).$$

与えられた$k_j$値の集合に対して静的MPF係数を決定するには、上記の2つの方程式で定義された変数$x_j$に対する線形方程式系を解く必要があります：$Ax=b$。ここで$x$は求めたい係数、$A$は$k_j$と使用するPFの種類（$S$）に依存する行列、$b$は制約のベクトルです。具体的には：

$$A_{0,j} = 1 $$
$$A_{i>0,j} = k_{j}^{-(\chi + s(i-1))}$$
$$b_0 = 1$$
$$b_{i>0} = 0 $$

ここで$\chi$は``order``、$s$は``symmetric``が``True``の場合は$2$、それ以外は$1$、$k_{j}$は``trotter_steps``、$x$は求める変数です。インデックス$i$と$j$は$0$から始まります。これを行列形式で視覚化することもできます：

$$
A =
\begin{bmatrix}
A_{0,0} & A_{0,1} & A_{0,2} & ... \\
A_{1,0} & A_{1,1} & A_{1,2} & ...  \\
A_{2,0} & A_{2,1} & A_{2,2} & ...  \\
... & ... & ... & ...
\end{bmatrix} =
\begin{bmatrix}
1 & 1 & 1 & ... \\
k_{0}^{-(\chi + s(1-1))} & k_{1}^{-(\chi + s(1-1))} & k_{2}^{-(\chi + s(1-1))} & ... \\
k_{0}^{-(\chi + s(2-1))} & k_{1}^{-(\chi + s(2-1))} & k_{2}^{-(\chi + s(2-1))} & ... \\
... & ... & ... & ...
\end{bmatrix}
$$

および

$$
b =
\begin{bmatrix}
b_{0} \\
b_{1} \\
b_{2}  \\
...
\end{bmatrix} =
\begin{bmatrix}
1 \\
0 \\
0  \\
...
\end{bmatrix}
$$

詳細については、線形方程式系（[LSE](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.LSE.html)）のドキュメントを参照してください。

$x$の解は$x = A^{-1}b$として解析的に求めることができます。例えば参考文献[\[1\]](#references)または[\[2\]](#references)を参照してください。
ただし、この厳密解は「悪条件」になる可能性があり、係数$x$のL1ノルムが非常に大きくなり、MPFの性能低下につながることがあります。
代わりに、$x$のL1ノルムを最小化する近似解を求めて、MPFの動作を最適化することも可能です。
##### LSEの設定
$k_j$値を選択したので、まず上記で説明したように、LSE（$Ax=b$）を構築する必要があります。
行列$A$は$k_j$だけでなく、PFの選択、特にその_次数_にも依存します。
さらに、`symmetric=True/False`を設定することで、PFが対称かどうかを考慮に入れることもできます（[\[1\]](#references)を参照）。
ただし、参考文献[\[2\]](#references)で示されているように、これは必須ではありません。

In [11]:
lse.b

array([1., 0., 0.])

上記で選択した値を使って$A$行列と$b$ベクトルの構築を確認しましょう。$j=0,1, 2$のTrotterステップ$k_j = [1, 2, 4]$、次数$\chi = 2$、非対称Trotterステップ（$s=1$）の選択により、最初の行より下の$A$の行列要素は式$A_{i>0,j} = k_{j}^{-(2 + 1(i-1))}$で決まります。具体的には：

$$ A_{0,0} = A_{0,1} = A_{0,2} =  1 $$
$$ A_{1,j} = k_{j}^{-1}  \rightarrow A_{1,0} = \frac{1}{1^2}, \;, A_{1,1} = \frac{1}{2^2}, \;, A_{1,2} = \frac{1}{4^2}$$
$$ A_{2,j} = k_{j}^{-2}  \rightarrow A_{2,0} = \frac{1}{1^3}, \;, A_{2,1} = \frac{1}{2^3}, \;, A_{2,2} = \frac{1}{4^3}$$

行列形式では：

$$
A =
\begin{bmatrix}
1 & 1 & 1\\
1 & \frac{1}{2^2} & \frac{1}{4^2}  \\
1 & \frac{1}{2^3} & \frac{1}{4^3}  \\
\end{bmatrix}
$$

これは`lse`オブジェクトを調べることで確認できます：

In [12]:
mpf_coeffs = lse.solve()
print(
    f"The static coefficients associated with the ansatze are: {mpf_coeffs}"
)

The static coefficients associated with the ansatze are: [ 0.04761905 -0.57142857  1.52380952]


##### Optimize for $x$ using an exact model

Alternatively to computing $x=A^{-1}b$, you can also use [setup_exact_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_exact_model.html) to construct a [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) instance that uses the LSE as constraints and whose optimal solution will yield $x$.

In the next section, it will be clear why this interface exists.

In [13]:
from qiskit_addon_mpf.costs import setup_exact_problem

model_exact, coeffs_exact = setup_exact_problem(lse)
model_exact.solve()
print(coeffs_exact.value)

[ 0.04761905 -0.57142857  1.52380952]


一方、制約ベクトル$b$は以下の要素を持ちます：
$$ b_{0} = 1 $$
$$ b_1 = b_2 = 0 $$

したがって、

$$
b =
\begin{bmatrix}
1 \\
0 \\
0
\end{bmatrix}
$$

同様に`lse`では：

In [14]:
print(
    "L1 norm of the exact coefficients:",
    np.linalg.norm(coeffs_exact.value, ord=1),
)  # ord specifies the norm. ord=1 is for L1

L1 norm of the exact coefficients: 2.1428571428556378


##### Optimize for $x$ using an approximate model

It might happen that the L1 norm for the chosen set of $k_j$ values is deemed too high.
If that is the case and you cannot choose a different set of $k_j$ values, you can use an approximate solution to the LSE instead of an exact one.

To do so, simply use [setup_approximate_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_approximate_model.html) to construct a different [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) instance, which constrains the L1-norm to a chosen threshold while minimizing the difference of $Ax$ and $b$.

In [16]:
from qiskit_addon_mpf.costs import setup_sum_of_squares_problem

model_approx, coeffs_approx = setup_sum_of_squares_problem(
    lse, max_l1_norm=1.5
)
model_approx.solve()
print(coeffs_approx.value)
print(
    "L1 norm of the approximate coefficients:",
    np.linalg.norm(coeffs_approx.value, ord=1),
)

[-1.10294118e-03 -2.48897059e-01  1.25000000e+00]
L1 norm of the approximate coefficients: 1.5


`lse`オブジェクトには、方程式系を満たす静的係数$x_j$を求めるためのメソッドがあります。

In [17]:
from qiskit_addon_utils.slicing import slice_by_depth
from qiskit_addon_mpf.backends.tenpy_layers import LayerModel
from qiskit_addon_mpf.backends.tenpy_layers import LayerwiseEvolver
from functools import partial

# Create approximate time-evolution circuits
single_2nd_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=order)
)
single_2nd_order_circ = pm.run(single_2nd_order_circ)  # collect XX and YY

# Find layers in the circuit
layers = slice_by_depth(single_2nd_order_circ, max_slice_depth=1)

# Create tensor network models
models = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz") for layer in layers
]

# Create the time-evolution object
approx_factory = partial(
    LayerwiseEvolver,
    layers=models,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 2,
    },
)

<Admonition type="warning">
The options of `LayerwiseEvolver` that determine the details of the tensor network simulation must be chosen carefully to avoid setting up an ill-defined optimization problem.
</Admonition>

We then set up the exact evolver (for example, [`ExactEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ExactEvolverFactory)), which returns an [`Evolver`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.html#qiskit_addon_mpf.backends.Evolver) object computing the true or “reference” time-evolution. Realistically, we would approximate the exact evolution by using a higher-order Suzuki–Trotter formula or another reliable method with a small time step. Below, we approximate the exact time-evolved state with a fourth-order Suzuki-Trotter formula using a small time step `dt=0.1`, which means the number of Trotter steps used at time $t$ is $k=t/dt$. We also specify some TeNPy-specific truncation options to bound the maximum bond dimension of the underlying tensor network, as well as the minimum singular values of the split tensor network bonds. These parameters can affect the accuracy of the expectation value calculated with the dynamic MPF coefficients, so it is important to explore a range of values to find the optimal balance between computational time and accuracy. Note that the calculation of the MPF coefficients does not rely of the expectation value of the PF obtained from hardware execution, and therefore it can be tuned in post-processing.

In [20]:
single_4th_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=4)
)
single_4th_order_circ = pm.run(single_4th_order_circ)
exact_model_layers = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz")
    for layer in slice_by_depth(single_4th_order_circ, max_slice_depth=1)
]

exact_factory = partial(
    LayerwiseEvolver,
    layers=exact_model_layers,
    dt=0.1,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 2,
    },
)

##### 厳密モデルを使用した$x$の最適化
$x=A^{-1}b$を計算する代わりに、[setup_exact_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_exact_model.html)を使用して、LSEを制約として使用し、その最適解が$x$を与える[cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem)インスタンスを構築することもできます。

次のセクションで、このインターフェースが存在する理由が明らかになります。

In [ ]:
from qiskit_addon_mpf.backends.tenpy_tebd import MPOState
from qiskit_addon_mpf.backends.tenpy_tebd import MPS_neel_state


def identity_factory():
    return MPOState.initialize_from_lattice(models[0].lat, conserve=True)


mps_initial_state = MPS_neel_state(models[0].lat)

For each time step $t$ we set up the dynamic linear system of equations with the [`setup_dynamic_lse`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html) method. The corresponding object contains the information about the dynamic MPF problem: `lse.A` gives the Gram matrix $M$ while `lse.b` gives the overlap $L$. We can then solve the LSE (when not ill-defined) to find the dynamic coefficients using the `setup_frobenius_problem`. It is important to note the difference with the static coefficients, which only depend on the details of the product formula used and are independent of the details of the time-evolution (Hamiltonian and initial state).

In [22]:
from qiskit_addon_mpf.dynamic import setup_dynamic_lse
from qiskit_addon_mpf.costs import setup_frobenius_problem

mpf_dynamic_coeffs_list = []
for t in trotter_times:
    print(f"Computing dynamic coefficients for time={t}")
    lse = setup_dynamic_lse(
        mpf_trotter_steps,
        t,
        identity_factory,
        exact_factory,
        approx_factory,
        mps_initial_state,
    )
    problem, coeffs = setup_frobenius_problem(lse)
    try:
        problem.solve()
        mpf_dynamic_coeffs_list.append(coeffs.value)
    except Exception as error:
        mpf_dynamic_coeffs_list.append(np.zeros(len(mpf_trotter_steps)))
        print(error, "Calculation Failed for time", t)
    print("")

Computing dynamic coefficients for time=0.5

Computing dynamic coefficients for time=0.6

Computing dynamic coefficients for time=0.7

Computing dynamic coefficients for time=0.7999999999999999

Computing dynamic coefficients for time=0.8999999999999999

Computing dynamic coefficients for time=0.9999999999999999

Computing dynamic coefficients for time=1.0999999999999999

Computing dynamic coefficients for time=1.1999999999999997

Computing dynamic coefficients for time=1.2999999999999998

Computing dynamic coefficients for time=1.4

Computing dynamic coefficients for time=1.4999999999999998



これらの係数で構築されたMPFが良い結果をもたらすかどうかの指標として、L1ノルムを使用できます（参考文献[\[1\]](#references)も参照）。

In [ ]:
import matplotlib.pyplot as plt

sym = {1: "^", 2: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    trotter_curve, trotter_curve_error = [], []
    for trotter_expvals, trotter_stds in zip(
        mpf_expvals_all_times, mpf_stds_all_times
    ):
        trotter_curve.append(trotter_expvals[k])
        trotter_curve_error.append(trotter_stds[k])

    plt.errorbar(
        trotter_times,
        trotter_curve,
        yerr=trotter_curve_error,
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )  # , , )

# Get expectation values at all times for the static MPF with exact coeffs
exact_mpf_curve, exact_mpf_curve_error = [], []
for trotter_expvals, trotter_stds in zip(
    mpf_expvals_all_times, mpf_stds_all_times
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(coeffs_exact.value, trotter_stds)
            ]
        )
    )
    exact_mpf_curve_error.append(mpf_std)
    exact_mpf_curve.append(trotter_expvals @ coeffs_exact.value)

plt.errorbar(
    trotter_times,
    exact_mpf_curve,
    yerr=exact_mpf_curve_error,
    markersize=4,
    marker="o",
    label="Static MPF - Exact",
    color="purple",
)


# Get expectation values at all times for the static MPF with approximate
approx_mpf_curve, approx_mpf_curve_error = [], []
for trotter_expvals, trotter_stds in zip(
    mpf_expvals_all_times, mpf_stds_all_times
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(coeffs_approx.value, trotter_stds)
            ]
        )
    )
    approx_mpf_curve_error.append(mpf_std)
    approx_mpf_curve.append(trotter_expvals @ coeffs_approx.value)

plt.errorbar(
    trotter_times,
    approx_mpf_curve,
    yerr=approx_mpf_curve_error,
    markersize=4,
    marker="o",
    color="orange",
    label="Static MPF - Approximate",
)

# # Get expectation values at all times for the dynamic MPF
dynamic_mpf_curve, dynamic_mpf_curve_error = [], []
for trotter_expvals, trotter_stds, dynamic_coeffs in zip(
    mpf_expvals_all_times, mpf_stds_all_times, mpf_dynamic_coeffs_list
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(dynamic_coeffs, trotter_stds)
            ]
        )
    )
    dynamic_mpf_curve_error.append(mpf_std)
    dynamic_mpf_curve.append(trotter_expvals @ dynamic_coeffs)

plt.errorbar(
    trotter_times,
    dynamic_mpf_curve,
    yerr=dynamic_mpf_curve_error,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)


plt.plot(
    exact_evolution_times,
    exact_expvals,
    lw=3,
    color="red",
    label="Exact time-evolution",
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) as a function of time"
)
plt.xlabel("Time")
plt.ylabel("Expectation Value")
plt.legend()
plt.grid()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/2da9c948-0.avif" alt="Output of the previous code cell" />

In the cases like the example above, where the $k=1$ PF behaves poorly at all times, the quality of the dynamic MPF results is also heavily affected. In such situations, it is useful to investigate the possibility of using individual PFs with higher number of Trotter steps to improve the overall quality of the results. In these simulations, we see the interplay of different types of errors: error from finite sampling, and Trotter error from the product formulas. MPF helps to reduce the Trotter error due to the product formulas but incur in higher sampling error compared to the product formulas. This can be advantageous, as product formulas can reduce the sampling error with increased sampling, but the systematic error due to the Trotter approximation remains untouched.

Another interesting behavior that we can observe from the plot is that the expectation value for the PF for $k=1$ starts to behave erratically (on top of not being a good approximation for the exact one) at times for which $t/k > 1 $, as explained in the [guide](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html) on how to choose the number of Trotter steps.

### Step 1: Map classical inputs to a quantum problem
Let's now consider a single time $t=1.0$ and calculate the expectation value of the magnetization with the various methods using one QPU. The particular choice of $t$ was done so to maximize the difference between the various methods and observe their relative efficacy. To determine the window of time for which dynamic MPF is guaranteed to produce observables with lower error than any of the individual Trotter formulas within the multi-product, we can implement the “MPF test” - see equation (17) and surrounding text in [\[3\]](#references).

#### Set up the Trotter circuits

At this point, we have found our expansion coefficients, $x$, and all that is left to do is to generate the Trotterized quantum circuits.
Once again, the [qiskit_addon_utils.problem_generators](/docs/api/qiskit-addon-utils/problem-generators) module comes to the rescue with a useful function to do this:

In [26]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


total_time = 1.0
mpf_circuits = []
for k in mpf_trotter_steps:
    # Initial Neel state preparation
    circuit = QuantumCircuit(L)
    circuit.x([i for i in range(L) if i % 2 != 0])

    trotter_circ = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(order=order, reps=k),
        time=total_time,
    )

    circuit.compose(trotter_circ, inplace=True)

    mpf_circuits.append(pm.run(circuit))

In [27]:
mpf_circuits[-1].draw("mpl", fold=-1, scale=0.4)

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/87d2ac0c-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum hardware execution
Let's return to the calculation of the expectation value for a single time point. We'll pick a backend for executing the experiment on hardware.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService


service = QiskitRuntimeService()
backend = service.least_busy(min_num_qubits=127)
print(backend)

qubits = list(range(backend.num_qubits))

この最適化問題の解法については完全な自由度があります。つまり、最適化ソルバー、その収束閾値などを変更できます。
[近似モデルの使用方法](https://qiskit.github.io/qiskit-addon-mpf/how_tos/using_approximate_model.html)に関するガイドを参照してください。
#### 動的MPF係数
前のセクションでは、標準的なTrotter近似を改善する静的MPFを導入しました。しかし、この静的バージョンは近似エラーを必ずしも最小化するわけではありません。具体的には、静的MPF（$\mu^S(t)$と表記）は、プロダクト公式状態${\rho_{k_i}(t)}_{i=1}^r$が張る部分空間への$\rho(t)$の最適な射影ではありません。

これに対処するため、フロベニウスノルムにおいて近似エラーを最小化する動的MPF（参考文献[\[2\]](#references)で導入され、参考文献[\[3\]](#references)で実験的に実証）を考えます。形式的には、以下を最小化することに焦点を当てます：

$$
\|\rho(t) - \mu^D(t)\|_F^2 \;=\; \mathrm{Tr}\bigl[ \left( \rho(t) - \mu^D(t)\right)^2 \bigr],
$$

各時刻$t$における係数$x_i(t)$に関して最小化します。フロベニウスノルムにおける*最適な*射影は$\mu^D(t) \;=\; \sum_{i=1}^r x_i(t)\,\rho_{k_i}(t)$であり、$\mu^D(t)$を*動的*MPFと呼びます。上記の定義を代入すると：

$$
\|\rho(t) - \mu^D(t)\|_F^2
\;=\; \\
= \mathrm{Tr}\bigl[ \left( \rho(t) - \mu^D(t)\right)^2 \bigr]
\;=\; \\
= \mathrm{Tr}\bigl[ \left( \rho(t) - \sum_{i=1}^r x_i(t)\,\rho_{k_i}(t) \right) \left(  \rho(t) - \sum_{j=1}^r x_j(t)\,\rho_{k_j}(t) \right) \bigr]
\;=\; \\
= 1 \;+\; \sum_{i,j=1}^r M_{i,j}(t)\,x_i(t)\,x_j(t)
\;-\;
2 \sum_{i=1}^r L_i^{\mathrm{exact}}(t)\,x_i(t),
$$

ここで$M_{i,j}(t)$は*グラム行列*であり、以下で定義されます：

$$
M_{i,j}(t) \;=\; \mathrm{Tr}\bigl[\rho_{k_i}(t)\,\rho_{k_j}(t)\bigr]
\;=\;
\bigl|\langle \psi_{\mathrm{in}} \!\mid S\bigl(t/k_i\bigr)^{-k_i}\,S\bigl(t/k_j\bigr)^{k_j} \!\mid \psi_{\mathrm{in}} \rangle \bigr|^2.
$$

および

$$
L_i^{\mathrm{exact}}(t) = \mathrm{Tr}[\rho(t)\,\rho_{k_i}(t)]
$$

は、厳密な状態$\rho(t)$と各プロダクト公式近似$\rho_{k_i}(t)$との間の重なりを表します。実用的なシナリオでは、ノイズや$\rho(t)$への部分的なアクセスにより、これらの重なりは近似的にしか測定できない場合があります。

ここで$\lvert\psi_{\mathrm{in}}\rangle$は初期状態、$S(\cdot)$はプロダクト公式で適用される操作です。この式を最小化する係数$x_i(t)$を選択し（$\rho(t)$が完全にはわからない場合の近似的な重なりデータを処理することで）、MPF部分空間内での$\rho(t)$の「最良の」（フロベニウスノルムの意味で）動的近似を得ます。量$L_i(t)$と$M_{i,j}(t)$はテンソルネットワーク法を使用して効率的に計算できます[\[3\]](#references)。MPF Qiskitアドオンは、この計算を実行するためのいくつかの「バックエンド」を提供しています。以下の例は最も柔軟な方法を示しており、[TeNPyレイヤーベースバックエンド](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.tenpy_layers.html#module-qiskit_addon_mpf.backends.tenpy_layers)のドキュメントでも詳細に説明されています。この方法を使用するには、所望の時間発展を実装する回路から始めて、対応する回路のレイヤーからこれらの操作を表すモデルを作成します。最後に、時間発展した量$M_{i,j}(t)$と$L_i(t)$を生成するために使用できる`Evolver`オブジェクトを作成します。まず、回路によって実装される近似的時間発展（[`ApproxEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ApproxEvolverFactory)）に対応する`Evolver`オブジェクトを作成します。特に、`order`変数が一致するように注意してください。近似的時間発展に対応する回路を生成する際には、`time = 1.0`とTrotterステップ数（`reps=1`）にプレースホルダー値を使用していることに注意してください。正しい近似回路は、`setup_dynamic_lse`の動的問題ソルバーによって生成されます。

In [ ]:
import copy
from qiskit.transpiler import Target, CouplingMap

target = backend.target
instruction_2q = "cz"

cmap = target.build_coupling_map(filter_idle_qubits=True)
cmap_list = list(cmap.get_edges())

max_meas_err = 0.012
min_t2 = 40
max_twoq_err = 0.005

# Remove qubits with bad measurement or t2
cust_cmap_list = copy.deepcopy(cmap_list)
for q in range(target.num_qubits):
    meas_err = target["measure"][(q,)].error
    if target.qubit_properties[q].t2 is not None:
        t2 = target.qubit_properties[q].t2 * 1e6
    else:
        t2 = 0
    if meas_err > max_meas_err or t2 < min_t2:
        # print(q)
        for q_pair in cmap_list:
            if q in q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue

# Remove qubits with bad 2q gate or t2
for q in cmap_list:
    twoq_gate_err = target[instruction_2q][q].error
    if twoq_gate_err > max_twoq_err:
        # print(q)
        for q_pair in cmap_list:
            if q == q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue


cust_cmap = CouplingMap(cust_cmap_list)

cust_target = Target.from_configuration(
    basis_gates=backend.configuration().basis_gates
    + ["measure"],  # or whatever new set of gates
    coupling_map=cust_cmap,
)

sorted_components = sorted(
    [list(comp.physical_qubits) for comp in cust_cmap.connected_components()],
    reverse=True,
)
print("size of largest component", len(sorted_components[0]))

size of largest component 10


> **Warning:** テンソルネットワークシミュレーションの詳細を決定する`LayerwiseEvolver`のオプションは、不適切に定義された最適化問題を設定しないよう、慎重に選択する必要があります。
次に、厳密なエボルバー（例：[`ExactEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ExactEvolverFactory)）を設定します。これは真の、あるいは「参照」時間発展を計算する[`Evolver`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.html#qiskit_addon_mpf.backends.Evolver)オブジェクトを返します。現実的には、高次の鈴木-Trotter公式や、小さな時間ステップを持つ別の信頼性の高い方法を使用して厳密な発展を近似します。以下では、小さな時間ステップ`dt=0.1`を用いた4次の鈴木-Trotter公式で厳密な時間発展状態を近似します。これは時刻$t$で使用されるTrotterステップ数が$k=t/dt$であることを意味します。また、基礎となるテンソルネットワークの最大ボンド次元および分割されたテンソルネットワークボンドの最小特異値を制限するための、TeNPy固有の切り捨てオプションも指定します。これらのパラメータは、動的MPF係数で計算される期待値の精度に影響を与える可能性があるため、計算時間と精度の最適なバランスを見つけるために、さまざまな値の範囲を探索することが重要です。MPF係数の計算はハードウェア実行から得られるPFの期待値に依存しないため、後処理で調整できることに注意してください。

In [34]:
cust_cmap.draw()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/c5d8e90b-0.avif" alt="Output of the previous code cell" />

次に、TeNPyと互換性のある形式でシステムの初期状態を作成します（例：`MPS_neel_state`=$\vert 0101...01 \rangle$）。これにより、テンソルとして時間発展させる多体波動関数$\lvert\psi_{\mathrm{in}}\rangle$が設定されます。

In [72]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

transpiler = generate_preset_pass_manager(
    optimization_level=3, target=cust_target
)

transpiled_circuits = [transpiler.run(circ) for circ in mpf_circuits]

qubits_layouts = [
    [
        idx
        for idx, qb in circuit.layout.initial_layout.get_physical_bits().items()
        if qb._register.name != "ancilla"
    ]
    for circuit in transpiled_circuits
]

transpiled_circuits = []
for circuit, layout in zip(mpf_circuits, qubits_layouts):
    transpiler = generate_preset_pass_manager(
        optimization_level=3, backend=backend, initial_layout=layout
    )
    transpiled_circuit = transpiler.run(circuit)
    transpiled_circuits.append(transpiled_circuit)


# transform the observable defined on virtual qubits to
# an observable defined on all physical qubits
isa_observables = [
    observable.apply_layout(circ.layout) for circ in transpiled_circuits
]

In [73]:
print(transpiled_circuits[-1].depth(lambda x: x.operation.num_qubits == 2))
print(transpiled_circuits[-1].count_ops())
transpiled_circuits[-1].draw("mpl", idle_wires=False, fold=False)

51
OrderedDict([('sx', 310), ('rz', 232), ('cz', 132), ('x', 19)])


<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/25ce07a6-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives
With the Estimator primitive we can obtain the estimation of expectation value from the QPU. We execute the optimized AQC circuits with additional error mitigation and suppression techniques.

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator


estimator = Estimator(mode=backend)
estimator.options.default_shots = 30000

# Set simple error suppression/mitigation options
estimator.options.dynamical_decoupling.enable = True
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.twirling.num_randomizations = "auto"
estimator.options.twirling.strategy = "active-accum"
estimator.options.resilience.measure_mitigation = True
estimator.options.experimental.execution_path = "gen3-turbo"

estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 3, 5)
estimator.options.resilience.zne.extrapolator = ("exponential", "linear")

estimator.options.environment.job_tags = ["mpf small"]


job = estimator.run(
    [
        (circ, observable)
        for circ, observable in zip(transpiled_circuits, isa_observables)
    ]
)

最後に、発展時間全体にわたってこれらの期待値をプロットします。

In [39]:
result_exp = job.result()
evs_exp = [res.data.evs for res in result_exp]
evs_std = [res.data.stds for res in result_exp]

print(evs_exp)

[array(-0.06361607), array(-0.23820448), array(-0.50271805)]


![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/2da9c948-0.avif)

上記の例のように、$k=1$のPFがすべての時間で性能が悪い場合、動的MPFの結果の質も大きく影響を受けます。このような状況では、結果の全体的な質を改善するために、より多くのTrotterステップを持つ個別のPFを使用する可能性を検討することが有用です。これらのシミュレーションでは、異なる種類のエラーの相互作用が見られます：有限サンプリングによるエラーと、プロダクト公式によるTrotterエラーです。MPFはプロダクト公式によるTrotterエラーを低減するのに役立ちますが、プロダクト公式と比較してより高いサンプリングエラーが発生します。プロダクト公式ではサンプリングを増やすことでサンプリングエラーを低減できますが、Trotter近似による系統的エラーは変わらないため、これは有利になり得ます。

プロットから観察できるもう1つの興味深い振る舞いは、$k=1$のPFの期待値が、$t/k > 1$となる時間において不規則な振る舞いを示し始めることです（良い近似でないことに加えて）。これはTrotterステップ数の選び方に関する[ガイド](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html)で説明されています。

### ステップ 1: 古典的な入力を量子問題にマッピングする
ここでは、単一の時刻 $t=1.0$ を考え、1つのQPUを使用してさまざまな方法で磁化の期待値を計算します。$t$ の値は、さまざまな方法の差異を最大化し、それぞれの相対的な有効性を観察できるように選択されています。動的MPFが、マルチプロダクト内の個々のトロッター公式のいずれよりも低い誤差で観測量を生成することが保証される時間の範囲を決定するために、「MPFテスト」を実装することができます。[\[3\]](#references) の式(17)およびその周辺のテキストを参照してください。

#### トロッター回路のセットアップ
この時点で、展開係数 $x$ が求まっており、あとはトロッター化された量子回路を生成するだけです。
ここでも、[qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) モジュールの便利な関数が役立ちます。

In [29]:
exact_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_exact.value, evs_std)
        ]
    )
)
print(
    "Exact static MPF expectation value: ",
    evs_exp @ coeffs_exact.value,
    "+-",
    exact_mpf_std,
)
approx_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_approx.value, evs_std)
        ]
    )
)
print(
    "Approximate static MPF expectation value: ",
    evs_exp @ coeffs_approx.value,
    "+-",
    approx_mpf_std,
)
dynamic_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(mpf_dynamic_coeffs_list[7], evs_std)
        ]
    )
)
print(
    "Dynamic MPF expectation value: ",
    evs_exp @ mpf_dynamic_coeffs_list[7],
    "+-",
    dynamic_mpf_std,
)

Exact static MPF expectation value:  -0.6329590442738475 +- 0.012798249760406036
Approximate static MPF expectation value:  -0.5690390035339492 +- 0.010459559917168473
Dynamic MPF expectation value:  -0.4655579758795695 +- 0.007639139186720507


Finally, for this small problem we can compute the exact reference value using [scipy.linalg.expm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.expm.html) as follows:

In [30]:
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

exp_H = expm(-1j * total_time * hamiltonian.to_matrix())

initial_state_circuit = QuantumCircuit(L)
initial_state_circuit.x([i for i in range(L) if i % 2 != 0])
initial_state = Statevector(initial_state_circuit).data

time_evolved_state = exp_H @ initial_state

exact_obs = (
    time_evolved_state.conj() @ observable.to_matrix() @ time_evolved_state
)
print("Exact expectation value ", exact_obs.real)

Exact expectation value  -0.39909900734489434


In [31]:
sym = {1: "^", 2: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    plt.errorbar(
        k,
        evs_exp[k],
        yerr=evs_std[k],
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )  # , , )


plt.errorbar(
    3,
    evs_exp @ coeffs_exact.value,
    yerr=exact_mpf_std,
    markersize=4,
    marker="o",
    color="purple",
    label="Static MPF",
)

plt.errorbar(
    4,
    evs_exp @ coeffs_approx.value,
    yerr=approx_mpf_std,
    markersize=4,
    marker="o",
    color="orange",
    label="Approximate static MPF",
)

plt.errorbar(
    5,
    evs_exp @ mpf_dynamic_coeffs_list[7],
    yerr=dynamic_mpf_std,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)

plt.axhline(
    y=exact_obs.real,
    linestyle="--",
    color="red",
    label="Exact time-evolution",
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) at time {total_time} for the different methods "
)
plt.xlabel("Method")
plt.ylabel("Expectation Value")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.grid(alpha=0.1)
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/a3eefe73-0.avif" alt="Output of the previous code cell" />

次に、トランスパイラのレイアウト段階で外れ値の量子ビットが含まれないように、カップリングマップから外れ値の量子ビットを除去します。以下では、`target` オブジェクトに格納されたバックエンドのプロパティ情報を使用し、測定誤差または2量子ビットゲートのエラーが特定の閾値（`max_meas_err`、`max_twoq_err`）を超えるか、$T_2$ 時間（コヒーレンスの損失を決定する）が特定の閾値（`min_t2`）を下回る量子ビットを除去します。

In [32]:
def relative_error(ev, exact_ev):
    return abs(ev - exact_ev)


relative_error_k = [relative_error(ev, exact_obs.real) for ev in evs_exp]
relative_error_mpf = relative_error(evs_exp @ mpf_coeffs, exact_obs.real)
relative_error_approx_mpf = relative_error(
    evs_exp @ coeffs_approx.value, exact_obs.real
)
relative_error_dynamic_mpf = relative_error(
    evs_exp @ mpf_dynamic_coeffs_list[7], exact_obs.real
)

print("relative error for each trotter steps", relative_error_k)
print("relative error with MPF exact coeffs", relative_error_mpf)
print("relative error with MPF approx coeffs", relative_error_approx_mpf)
print("relative error with MPF dynamic coeffs", relative_error_dynamic_mpf)

relative error for each trotter steps [0.33548293650112293, 0.16089452939226306, 0.10361904247828346]
relative error with MPF exact coeffs 0.2338600369291003
relative error with MPF approx coeffs 0.16993999618905486
relative error with MPF dynamic coeffs 0.06645896853467514


## Part II: scale it up

Let's scale the problem up beyond what is possible to simulate exactly. In this section we will focus on reproducing some of the results shown in Ref. [\[3\]](#references).

### Step 1: Map classical inputs to a quantum problem

#### Hamiltonian

For the large-scale example, we use the XXZ model on a line of 50 sites:

$$
\hat{\mathcal{H}}_{XXZ} = \sum_{i=1}^{L-1} J_{i,(i+1)}\left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ 2\cdot Z_i Z_{(i+1)} \right) \, ,
$$

where $J_{i,(i+1)}$ is a random coefficient corresponding to edge $(i, i+1)$. This is the Hamiltonian considered in the demonstration presented in Ref. [\[3\]](#references).

In [33]:
L = 50
# Generate some coupling map to use for this example
coupling_map = CouplingMap.from_line(L, bidirectional=False)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/34bf68ac-0.avif" alt="Output of the previous code cell" />

In [ ]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp, Pauli


# Generate random coefficients for our XXZ Hamiltonian
np.random.seed(0)
even_edges = list(coupling_map.get_edges())[::2]
odd_edges = list(coupling_map.get_edges())[1::2]

Js = np.random.uniform(0.5, 1.5, size=L)
hamiltonian = SparsePauliOp(Pauli("I" * L))
for i, edge in enumerate(even_edges + odd_edges):
    hamiltonian += SparsePauliOp.from_sparse_list(
        [
            ("XX", (edge), 2 * Js[i]),
            ("YY", (edge), 2 * Js[i]),
            ("ZZ", (edge), 4 * Js[i]),
        ],
        num_qubits=L,
    )

print(hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXX', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYY', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIIII', 'IIIIIIIIIIII

![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/c5d8e90b-0.avif)

次に、回路と観測量をデバイスの物理量子ビットにマッピングします。

In [35]:
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L
)
print(observable)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIIIIIIIIIIIIIIIIII'],
              coeffs=[1.+0.j])


#### Choose Trotter steps
The experiment showcased in Fig. 4 of Ref. [\[3\]](#references) uses $k_j = [2, 3, 4]$ symmetric Trotter steps of order $2$. We focus on the results for time $t=3$, where the MPF and a PF with a higher number of Trotter steps (6 in this case) have the same Trotter error. However, the MPF expectation value is calculated from circuits corresponding to the lower number of Trotter steps and thus shallower. In practice, even if the MPF and the deeper Trotter steps circuit have the same Trotter error, we expect the experimental expectation value calculated from the MPF circuits to be closer to the theory one, as it entails running shallower circuits less exposed to hardware noise compared to the circuit corresponding to the higher Trotter step PF.

In [36]:
total_time = 3
mpf_trotter_steps = [2, 3, 4]
order = 2
symmetric = True

#### Set up the LSE
Here we look at the static MPF coefficients for this problem.

In [37]:
lse = setup_static_lse(mpf_trotter_steps, order=order, symmetric=symmetric)
mpf_coeffs = lse.solve()
print(
    f"The static coefficients associated with the ansatze are: {mpf_coeffs}"
)
print("L1 norm:", np.linalg.norm(mpf_coeffs, ord=1))

The static coefficients associated with the ansatze are: [ 0.26666667 -2.31428571  3.04761905]
L1 norm: 5.628571428571431


In [38]:
model_approx, coeffs_approx = setup_sum_of_squares_problem(
    lse, max_l1_norm=2.0
)
model_approx.solve()
print(coeffs_approx.value)
print(
    "L1 norm of the approximate coefficients:",
    np.linalg.norm(coeffs_approx.value, ord=1),
)

[-0.24255546 -0.25744454  1.5       ]
L1 norm of the approximate coefficients: 2.0


### ステップ 4: 後処理と所望の古典形式での結果の返却
唯一の後処理ステップは、Qiskit Runtimeプリミティブから異なるトロッターステップで得られた期待値を、それぞれのMPF係数を使用して結合することです。観測量 $A$ に対して、次のように表されます：

$$ \langle A \rangle_{\text{mpf}}  = \text{Tr} [A \mu(t)] = \sum_{j} x_j  \text{Tr} [A \rho_{k_j}] = \sum_{j} x_j \langle A \rangle_j$$

まず、各トロッター回路から得られた個々の期待値を抽出します。

In [ ]:
# Create approximate time-evolution circuits
single_2nd_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=order)
)
single_2nd_order_circ = pm.run(single_2nd_order_circ)  # collect XX and YY

# Find layers in the circuit
layers = slice_by_depth(single_2nd_order_circ, max_slice_depth=1)

# Create tensor network models
models = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz") for layer in layers
]

# Create the time-evolution object
approx_factory = partial(
    LayerwiseEvolver,
    layers=models,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 4,
    },
)

# Create exact time-evolution circuits
single_4th_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=4)
)
single_4th_order_circ = pm.run(single_4th_order_circ)
exact_model_layers = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz")
    for layer in slice_by_depth(single_4th_order_circ, max_slice_depth=1)
]

# Create the time-evolution object
exact_factory = partial(
    LayerwiseEvolver,
    layers=exact_model_layers,
    dt=0.1,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 3,
    },
)


def identity_factory():
    return MPOState.initialize_from_lattice(models[0].lat, conserve=True)


mps_initial_state = MPS_neel_state(models[0].lat)


lse = setup_dynamic_lse(
    mpf_trotter_steps,
    total_time,
    identity_factory,
    exact_factory,
    approx_factory,
    mps_initial_state,
)
problem, coeffs = setup_frobenius_problem(lse)
try:
    problem.solve()
    mpf_dynamic_coeffs = coeffs.value
except Exception as error:
    print(error, "Calculation Failed for time", total_time)
print("")

#### Construct each of the Trotter circuits in our MPF decomposition

In [41]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


mpf_circuits = []
for k in mpf_trotter_steps:
    # Initial state preparation |1010..>
    circuit = QuantumCircuit(L)
    circuit.x([i for i in range(L) if i % 2])

    trotter_circ = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(reps=k, order=order),
        time=total_time,
    )

    circuit.compose(trotter_circ, qubits=range(L), inplace=True)

    mpf_circuits.append(circuit)

次に、MPF係数を使用してこれらを再結合し、MPFの総期待値を求めます。以下では、$x$ を計算したさまざまな方法のそれぞれについてこれを行います。

In [42]:
k = 6

# Initial state preparation |1010..>
comp_circuit = QuantumCircuit(L)
comp_circuit.x([i for i in range(L) if i % 2])


trotter_circ = generate_time_evolution_circuit(
    hamiltonian,
    synthesis=SuzukiTrotter(reps=k, order=order),
    time=total_time,
)

comp_circuit.compose(trotter_circ, qubits=range(L), inplace=True)


mpf_circuits.append(comp_circuit)

### Step 2: Optimize problem for quantum hardware execution

In [ ]:
import copy
from qiskit.transpiler import Target, CouplingMap

target = backend.target
instruction_2q = "cz"

cmap = target.build_coupling_map(filter_idle_qubits=True)
cmap_list = list(cmap.get_edges())

max_meas_err = 0.055
min_t2 = 30
max_twoq_err = 0.01

# Remove qubits with bad measurement or t2
cust_cmap_list = copy.deepcopy(cmap_list)
for q in range(target.num_qubits):
    meas_err = target["measure"][(q,)].error
    if target.qubit_properties[q].t2 is not None:
        t2 = target.qubit_properties[q].t2 * 1e6
    else:
        t2 = 0
    if meas_err > max_meas_err or t2 < min_t2:
        # print(q)
        for q_pair in cmap_list:
            if q in q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue

# Remove qubits with bad 2q gate or t2
for q in cmap_list:
    twoq_gate_err = target[instruction_2q][q].error
    if twoq_gate_err > max_twoq_err:
        # print(q)
        for q_pair in cmap_list:
            if q == q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue


cust_cmap = CouplingMap(cust_cmap_list)

cust_target = Target.from_configuration(
    basis_gates=backend.configuration().basis_gates
    + ["measure"],  # or whatever new set of gates
    coupling_map=cust_cmap,
)

sorted_components = sorted(
    [list(comp.physical_qubits) for comp in cust_cmap.connected_components()],
    reverse=True,
)
print("size of largest component", len(sorted_components[0]))

size of largest component 73


In [104]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

transpiler = generate_preset_pass_manager(
    optimization_level=3, target=cust_target
)

transpiled_circuits = [transpiler.run(circ) for circ in mpf_circuits]

qubits_layouts = [
    [
        idx
        for idx, qb in circuit.layout.initial_layout.get_physical_bits().items()
        if qb._register.name != "ancilla"
    ]
    for circuit in transpiled_circuits
]

transpiled_circuits = []
for circuit, layout in zip(mpf_circuits, qubits_layouts):
    transpiler = generate_preset_pass_manager(
        optimization_level=3, backend=backend, initial_layout=layout
    )
    transpiled_circuit = transpiler.run(circuit)
    transpiled_circuits.append(transpiled_circuit)


# transform the observable defined on virtual qubits to
# an observable defined on all physical qubits
isa_observables = [
    observable.apply_layout(circ.layout) for circ in transpiled_circuits
]

### Step 3: Execute using Qiskit primitives

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode=backend)
estimator.options.default_shots = 30000

# Set simple error suppression/mitigation options
estimator.options.dynamical_decoupling.enable = True
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.twirling.num_randomizations = "auto"
estimator.options.twirling.strategy = "active-accum"
estimator.options.resilience.measure_mitigation = True
estimator.options.experimental.execution_path = "gen3-turbo"

estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 1.2, 1.4)
estimator.options.resilience.zne.extrapolator = "linear"

estimator.options.environment.job_tags = ["mpf large"]

job_50 = estimator.run(
    [
        (circ, observable)
        for circ, observable in zip(transpiled_circuits, isa_observables)
    ]
)

### Step 4: Post-process and return result in desired classical format

In [46]:
result = job_50.result()
evs = [res.data.evs for res in result]
std = [res.data.stds for res in result]

print(evs)
print(std)

[array(-0.08034071), array(-0.00605026), array(-0.15345759), array(-0.18127293)]
[array(0.04482517), array(0.03438413), array(0.21540776), array(0.21520829)]


In [52]:
exact_mpf_std = np.sqrt(
    sum([(coeff**2) * (std**2) for coeff, std in zip(mpf_coeffs, std[:3])])
)
print(
    "Exact static MPF expectation value: ",
    evs[:3] @ mpf_coeffs,
    "+-",
    exact_mpf_std,
)
approx_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_approx.value, std[:3])
        ]
    )
)
print(
    "Approximate static MPF expectation value: ",
    evs[:3] @ coeffs_approx.value,
    "+-",
    approx_mpf_std,
)
dynamic_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(mpf_dynamic_coeffs, std[:3])
        ]
    )
)
print(
    "Dynamic MPF expectation value: ",
    evs[:3] @ mpf_dynamic_coeffs,
    "+-",
    dynamic_mpf_std,
)

Exact static MPF expectation value:  -0.47510243192011536 +- 0.6613940032465087
Approximate static MPF expectation value:  -0.20914170384216998 +- 0.32341567460419135
Dynamic MPF expectation value:  -0.07994951978722761 +- 0.07423091963310202


In [ ]:
sym = {2: "^", 3: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    plt.errorbar(
        k,
        evs[k],
        yerr=std[k],
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )


plt.errorbar(
    3,
    evs[-1],
    yerr=std[-1],
    alpha=0.5,
    markersize=8,
    marker="x",
    color="blue",
    label="6 Trotter steps",
)


plt.errorbar(
    4,
    evs[:3] @ mpf_coeffs,
    yerr=exact_mpf_std,
    markersize=4,
    marker="o",
    color="purple",
    label="Static MPF",
)

plt.errorbar(
    5,
    evs[:3] @ coeffs_approx.value,
    yerr=approx_mpf_std,
    markersize=4,
    marker="o",
    color="orange",
    label="Approximate static MPF",
)

plt.errorbar(
    6,
    evs[:3] @ mpf_dynamic_coeffs,
    yerr=dynamic_mpf_std,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)

exact_obs = -0.24384471447172074  # Calculated via Tensor Network calculation
plt.axhline(
    y=exact_obs, linestyle="--", color="red", label="Exact time-evolution"
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) at time {total_time} for the different methods "
)
plt.xlabel("Method")
plt.ylabel("Expectation Value")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.grid(alpha=0.1)
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/d751af7c-0.avif" alt="Output of the previous code cell" />

## パート II: スケールアップ
厳密にシミュレーションすることが不可能な規模まで問題をスケールアップしましょう。このセクションでは、参考文献 [\[3\]](#references) に示されている結果の一部を再現することに焦点を当てます。
### ステップ 1: 古典的な入力を量子問題にマッピングする
#### ハミルトニアン
大規模な例として、50サイトの直線上のXXZモデルを使用します：

$$
\hat{\mathcal{H}}_{XXZ} = \sum_{i=1}^{L-1} J_{i,(i+1)}\left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ 2\cdot Z_i Z_{(i+1)} \right) \, ,
$$

ここで $J_{i,(i+1)}$ はエッジ $(i, i+1)$ に対応するランダムな係数です。これは参考文献 [\[3\]](#references) で示されたデモンストレーションで考慮されたハミルトニアンです。